## 装饰器

In [27]:
import collections
from functools import wraps

In [28]:
function_called_time = collections.defaultdict(int)

def get_call_time(func):
    @wraps(func)
    def _inner(*args, **kwargs):
        """It's inner function"""
        global function_called_time
        function_called_time[func.__name__] += 1
        result = func(*args, **kwargs)
        print('function called time is : {}'.format(function_called_time[func.__name__]))
        return result
    return _inner

In [29]:
@get_call_time
def func_1(n):
    """
    @param n: is the number of customers
    @return int: the customers value point
    """
    for i in range(n):
        print(n)
    return 0

In [37]:
func_1(5)

5
5
5
5
5
function called time is : 3


0

In [31]:
help(func_1)

Help on function func_1 in module __main__:

func_1(n)
    @param n: is the number of customers
    @return int: the customers value point



## Dynamic Programming For Cutting Problems

In [2]:
from collections import defaultdict

original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30]
price = defaultdict(int)
for i, p in enumerate(original_price):
    price[i+1] = p
assert price[1] == 1

In [14]:
solution = {}

In [48]:
from functools import wraps

def memo(func):
    cache = {}
    @wraps(func)
    def _wrap(*args, **kwargs):
        if args in cache:
            result = cache[args]
        else:
            result = func(*args, **kwargs)
            cache[args] = result
        return result
    return _wrap

In [49]:
@memo
def r(n):
    max_price, split_point = max([(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key=lambda x: x[0])
    solution[n] = (split_point, n - split_point)
    return max_price

In [50]:
r(20)

60

In [54]:
solution

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10),
 11: (1, 10),
 12: (2, 10),
 13: (3, 10),
 14: (2, 12),
 15: (2, 13),
 16: (6, 10),
 17: (1, 16),
 18: (2, 16),
 19: (3, 16),
 20: (10, 10)}

In [51]:
def parse_solution(target_length, revenue_solution):
    left, right = revenue_solution[target_length]
    
    if left == 0: return [right]

    return parse_solution(left, revenue_solution) + parse_solution(right, revenue_solution)

In [52]:
parse_solution(20, solution)

[10, 10]

In [53]:
parse_solution(17, solution)

[1, 6, 10]

## Edit Distance

In [59]:
solution = {}

# @lru_cache(maxsize=2**10)
@memo
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation 
    
    return min_distance

In [60]:
edit_distance('ABCDE', 'ABCCEF')

2

In [61]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F'}

## Part1-2: Finish the Solution Parse Part of Edit-Distance

In [82]:
def parse_solution_of_ED(str1, str2, revenue_solution):
    if str1 == str2:
        return []
    
    action = revenue_solution[(str1, str2)]
    if not action:
        return parse_solution_of_ED(str1[:-1], str2[:-1],solution)
    op = action.split()[0]
    if op == 'ADD':
        return parse_solution_of_ED(str1, str2[:-1],solution) + [action]
    elif op == 'DEL':
        return parse_solution_of_ED(str1[:-1], str2,solution) + [action]
    else:
        return parse_solution_of_ED(str1[:-1], str2[:-1],solution) + [action]
    
parse_solution_of_ED('ABCDE', 'ABCCEF',solution)

['SUB D => C', 'ADD F']